In [1]:
# psavert - personal savings rate.
# pce - personal consumption expenditures, in billions of dollars.
# uempmed - median duration of unemployment, in weeks.
# pop - total population, in thousands.
# unemploy- number of unemployed in thousands (dependent variable).

In [2]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

In [3]:
df1 = pd.read_csv('PSAVERT.csv') 

In [4]:
df2 = pd.read_csv('PCE.csv')

In [5]:
df3 = pd.read_csv('UEMPMED.csv')

In [6]:
df4 = pd.read_csv('POP.csv')

In [7]:
df_target = pd.read_csv('UNEMPLOY.csv') 

In [8]:
df5 = pd.merge(df1,df2,how="outer", on="DATE")

In [9]:
df6 = pd.merge(df3,df4, how="outer", on="DATE")

In [10]:

df7 = pd.merge(df5,df6,how='left',on='DATE')
df_final = pd.merge(df7,df_target,how='left',on='DATE')
df_final.head()

,DATE,PSAVERT,PCE,UEMPMED,POP,UNEMPLOY
0,1959-01-01,11.3,306.1,NaN,176447.0,4068
1,1959-02-01,10.6,309.6,NaN,176685.0,3965
2,1959-03-01,10.3,312.7,NaN,176905.0,3801
3,1959-04-01,11.2,312.2,NaN,177146.0,3571
4,1959-05-01,10.6,316.1,NaN,177365.0,3479


In [11]:
df = df_final.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 659 entries, 102 to 760
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   DATE      659 non-null    object 
 1   PSAVERT   659 non-null    float64
 2   PCE       659 non-null    float64
 3   UEMPMED   659 non-null    float64
 4   POP       659 non-null    float64
 5   UNEMPLOY  659 non-null    int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 36.0+ KB


In [12]:

df.to_csv('unemployment.csv',index = False)

In [13]:
del df['DATE']

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df)

MinMaxScaler()

In [15]:
df.describe()


,PSAVERT,PCE,UEMPMED,POP,UNEMPLOY
count,659.000000,659.000000,659.000000,659.000000,659.000000
mean,8.576935,6015.381639,8.943399,266227.850868,7799.455235
std,3.322007,4562.388545,4.106245,41592.043310,2696.516948
min,2.100000,506.700000,1.500000,198712.000000,2685.000000
25%,6.400000,1903.650000,6.200000,229313.500000,6270.500000
50%,8.100000,4841.600000,8.100000,264804.000000,7468.000000
75%,11.000000,9941.850000,9.650000,305177.000000,8624.000000
max,33.800000,16956.600000,25.200000,332884.622000,23038.000000


In [16]:
df


,PSAVERT,PCE,UEMPMED,POP,UNEMPLOY
102,12.6,506.7,4.5,198712.000,2944
103,12.6,509.8,4.7,198911.000,2945
104,11.9,515.6,4.6,199113.000,2958
105,12.9,512.2,4.9,199311.000,3143
106,12.8,517.4,4.7,199498.000,3066
...,...,...,...,...,...
756,5.8,16543.3,10.1,332650.428,6513
757,5.8,16635.8,9.6,332717.925,6270
758,5.3,16831.2,7.5,332782.634,5952
759,5.2,16923.9,7.5,332842.301,5941


In [17]:
target_column = ['UNEMPLOY'] 
predictors = list(set(list(df.columns))-set(target_column))
df[predictors] = df[predictors]/df[predictors].max()
df.describe()

C:\Anaconda\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,PSAVERT,PCE,UEMPMED,POP,UNEMPLOY
count,659.000000,659.000000,659.000000,659.000000,659.000000
mean,0.253755,0.354752,0.354897,0.799760,7799.455235
std,0.098284,0.269063,0.162946,0.124944,2696.516948
min,0.062130,0.029882,0.059524,0.596940,2685.000000
25%,0.189349,0.112266,0.246032,0.688868,6270.500000
50%,0.239645,0.285529,0.321429,0.795483,7468.000000
75%,0.325444,0.586312,0.382937,0.916765,8624.000000
max,1.000000,1.000000,1.000000,1.000000,23038.000000


In [18]:
X = df[predictors].values
y = df[target_column].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print(X_train.shape); print(X_test.shape)

(461, 4)
(198, 4)


In [19]:
#LinearRegression

In [20]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [21]:
pred_train_lr= lr.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train_lr)))
print(r2_score(y_train, pred_train_lr))

pred_test_lr= lr.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test_lr))) 
print(r2_score(y_test, pred_test_lr))

1197.1953365048298
0.8126665147535064
1181.7990900145478
0.7765420685274688


In [22]:
#Ridge Regression
#Ridge

In [23]:
rr = Ridge(alpha=0.01)
rr.fit(X_train, y_train) 
pred_train_rr= rr.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print(r2_score(y_train, pred_train_rr))

pred_test_rr= rr.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print(r2_score(y_test, pred_test_rr))

1199.2186422989923
0.8120327782117314
1187.911759741384
0.7742244883917245


In [24]:
#
#model_lasso

In [25]:
model_lasso = Lasso(alpha=0.01)
model_lasso.fit(X_train, y_train) 
pred_train_lasso= model_lasso.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train_lasso)))
print(r2_score(y_train, pred_train_lasso))

pred_test_lasso= model_lasso.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test_lasso))) 
print(r2_score(y_test, pred_test_lasso))

1197.1955933220454
0.8126664343815441
1181.8591194494652
0.7765193668785156


In [26]:
#ElasticNet Regression
# made by combining ridge & lasso

In [27]:
#Elastic Net
model_enet = ElasticNet(alpha = 0.01)
model_enet.fit(X_train, y_train) 
pred_train_enet= model_enet.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train_enet)))
print(r2_score(y_train, pred_train_enet))

pred_test_enet= model_enet.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test_enet)))
print(r2_score(y_test, pred_test_enet))

1670.4846877262494
0.6352709307893849
1685.5339631453367
0.5454480894340827


In [28]:
#Deep Learning Regression Model

In [29]:
# Keras specific
import keras
from keras.models import Sequential
from keras.layers import Dense

In [30]:

model = Sequential()
model.add(Dense(500, input_dim=4, activation= "relu"))
model.add(Dense(100, activation= "relu"))
model.add(Dense(50, activation= "relu"))
model.add(Dense(1))
model.summary() #Print model Summary

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 500)               2500      
                                                                 
 dense_1 (Dense)             (None, 100)               50100     
                                                                 
 dense_2 (Dense)             (None, 50)                5050      
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 57,701
Trainable params: 57,701
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
15/15 [==============================] - 1s 2ms/step - loss: 70258080.0000 - mean_squared_error: 70258080.0000
Epoch 2/20
15/15 [==============================] - 0s 2ms/step - loss: 70198080.0000 - mean_squared_error: 70198080.0000
Epoch 3/20
15/15 [==============================] - 0s 2ms/step - loss: 69983088.0000 - mean_squared_error: 69983088.0000
Epoch 4/20
15/15 [==============================] - 0s 2ms/step - loss: 69361736.0000 - mean_squared_error: 69361736.0000
Epoch 5/20
15/15 [==============================] - 0s 2ms/step - loss: 67869560.0000 - mean_squared_error: 67869560.0000
Epoch 6/20
15/15 [==============================] - 0s 2ms/step - loss: 64762096.0000 - mean_squared_error: 64762096.0000
Epoch 7/20
15/15 [==============================] - 0s 2ms/step - loss: 59126660.0000 - mean_squared_error: 59126660.0000
Epoch 8/20
15/15 [==============================] - 0s 2ms/step - loss: 50253792.0000 - mean_squared_error: 50253792.0000
Epoch 9/20
15/15 [======

In [32]:
pred_train= model.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train)))

pred= model.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred))) 

2148.329279448318
2047.172995705839


In [33]:

print(r2_score(y_train, pred_train))

print(r2_score(y_test, pred))

0.39676424509908925
0.3294710085967977
